# Assignment 1 - Part 2
Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 1 part 2 <br>
Date: 10/26/2021 <br>
Group Members from Assignment 5: Emma Bright, Mike Santoro <br>

Association Rules were originally created for mining information for market baskets.  But, they can provide insight into
 other applications also.  In this part of the assignment you will use the mlxtend library functions apriori( )
  and association_rules( ) to explore text data.  Specifically, we will look at the text from three books:

- Sense and Sensibility, by Jane Austin, published 1811
- A Tale of Two Cities, by Charles Dickens, published 1859
- On The Origin of Species, by Charles Darwin, published 1859 (although written over a 20 year period preceding that
date).

All three books were written in english by english people at "roughly" the same time, hence one could assume some
commonality in writing style.

Mining using association rules assumes a database of a large number of transactions with some number of items per
transaction.  I have processed the text of these books to be presented in a similar fashion, where each "transaction"
is now a sentence from the book with the words comma-separated.  Note, some of the sentences in the books are quite
long which would result in intractable run times (and enormous space to hold all possible itemsets). As we saw from
our reading, the number of subsets to be considered increases exponentially with the number of items per transaction.
 For example, assume a sentence has 60 words.   Then we might need to consider all possible subsets of size 60, 59, 58,
  57, ... and 1. The number of these would be:  Choose(60,60) + Choose(60,59) + Choose(60,58) + ...Choose(60,2) +
  Choose(60,1).  Clearly not tractable!

To make this problem tractable I have processed the texts to the they contain at most N words per line (transaction),
 where N = {10,12,14}.  For example, assume N = 14.  I processed the file into a list of sentences.  Then, for each
 sentence, if there were more than 14 words I cut into into at most 14 work chunks.  The beginning of a new sentence
 starts a new line.  I also removed all punctuation and made everything lower case.  As far as text analysis goes,
 this loses some useful information, but it greatly simplifies the problem and this assignment is more about
  understanding how to use association rules and frequent itemsets.

On my computer the files for N=14 ran very quickly, about 5 seconds or less depending on the min-support value used,
 but I have provided the smaller sets just in case you need to use them due to having a slower computer. To figure out
 which set of files to use, I suggest you run your code on the 10 words per line, 12 words per line, and 14 words per
  line data sets in turn to see how fast they run.  If you computer works well on the 14, only use the 14 for all three
   texts.  If you need to use the 10 or 12, only use that value for all three texts.

You are to explore each of the three texts and see what you can determine about the texts individually, and compared to
 each other.  Write up a summary of commonality and differences explaining the parameter values used.  I suggest you
  look at frequent 1-itemsets,  2-itemsets and possibly 3-itemsets  as well as association rules.  As a starting point
  I suggest you start with a min-support value of 0.02 and a min-confidence value of 0.2.  Then increase/decrease both
   as you see fit.  Only consider those association rules that have a lift value > someValue, where someValue is around
    1.1 .. 1.5.

The same values may not work well for each of the datasets, it is fine to use different values for different texts,
just make sure to explain what values you used and whey.

Write up your experiments as a report and upload the report as a .pdf file.  Also attach your code for one of your
experiments (all the others should be quite similar).  You do not need one file that runs everything, how much you
"automate" via python scripting is up to you.

In [213]:
# Adding in all the import statements
from csv import reader
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', None)

In [214]:
# importing the texts
def fill_book_list(file_name):
    """This function reads in the book"""
    book = []
    with open(file_name, 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            book.append(row)
    return book

book1 = fill_book_list('input_SenseAndSensibility_14.csv')
book2 = fill_book_list('input_TaleOfTwoCities_14.csv')
book3 = fill_book_list('input_OriginOfSpecies_14.csv')

Creating a function for using the transaction encoder to transform the books into an 1-hot boolean encoded numpy array
and convert into a dataframe for the convenience of passing into apriori

In [215]:
def transact_1_hot(book):
    """Creating a function to easily run the apriori and edit the min_support, min_threshold, and confidence"""
    te = TransactionEncoder()
    te_ary = te.fit(book).transform(book)
    df = pd.DataFrame(te_ary, columns= te.columns_)
    return df

book1_df = transact_1_hot(book1)
book2_df = transact_1_hot(book2)
book3_df = transact_1_hot(book3)

Displaying the head of all three books

In [216]:
book1_df.head()

,200,7000l,_,_a,_all,_almost_,_always_,_am_,_amiably,_and,...,youthere,youthful,youtravel,youwill,youwilloughby,youyet,zeal,zealous,zealously,Ã
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [217]:
book2_df.head()

,1757,1767,211792,_,_am_,_are_collected,_at_,_bid_,_do_,_don,...,youso,youth,youthful,youthfulness,youths,youwell,youwill,youworth,youyes,zealous
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [218]:
book3_df.head()

,000,1,10,100,1000,101,102,107,107330,107416,...,zealand,zealand100,zealand201,zealand376,zebra,zero,zone,zones,zoological,zoologist
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Creating a function to run the apriori algorithm

In [219]:
def run_apriori(df, min_support_feed=0.02, min_threshold_feed=0.2):
    """This function runs through the apriori algorithm"""
    freq_items = apriori(df, min_support=min_support_feed, use_colnames=True)
    freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
    freq_items.sort_values(["support", "length"], inplace=True, ascending=False)
    rules = association_rules(freq_items, metric="confidence", min_threshold=min_threshold_feed)
    return rules, freq_items

book1_rules, book1_freq_items = run_apriori(book1_df)
book2_rules, book2_freq_items = run_apriori(book2_df)
book3_rules, book3_freq_items = run_apriori(book3_df)

Displaying the head of all three items, then rules

In [220]:
# book1_freq_items.head()

In [221]:
# book1_rules.head()

In [222]:
# book2_freq_items.head()

In [223]:
# book2_rules.head()

In [224]:
# book3_freq_items.head()

In [225]:
# book3_rules.head()

Taking a look at frequent 1-itemsets that are both strong and interesting

In [226]:
def filter_freq_items(freq_items,items=1):
    """Looking at string and interesting rules"""
    filtered_freq_items = freq_items[freq_items["length"] == items].copy()
    return filtered_freq_items

The five most common one words for every book with a min-support of 0.02

In [227]:
book1_freq_items_1 = filter_freq_items(book1_freq_items)
book1_freq_items_1.head(5)

,support,itemsets,length
61,0.300081,(to),1
56,0.283155,(the),1
4,0.276672,(and),1
42,0.269650,(of),1
22,0.174845,(her),1


In [228]:
book2_freq_items_1 = filter_freq_items(book2_freq_items)
book2_freq_items_1.head(5)

,support,itemsets,length
44,0.421277,(the),1
3,0.324901,(and),1
34,0.259023,(of),1
50,0.232467,(to),1
0,0.192829,(a),1


In [229]:
book3_freq_items_1 = filter_freq_items(book3_freq_items)
book3_freq_items_1.head(5)

,support,itemsets,length
50,0.522363,(the),1
36,0.434390,(of),1
3,0.283951,(and),1
25,0.257707,(in),1
56,0.225992,(to),1


### Frequent 1-itemsets
When we look at the frequent item sets for 1 word items from the three books we notice a lot of similarities.
All three books have the following words in the top 5;
- The
- To
- And
- Of <br>

Books two and three has a really high support for the word "the". The rankings of the top 5 words varies but there is
Nothing that is too surprising here. I could see Charles Darwin using "The" and "Of" almost every sentence trying to talk
about the origin of the species. <br>

Now looking into the differences we do find that the support ranking order is different. Listed below are the words that
 appreared in the top 5 for one book but not the others:
- "her" (book1)
- "a" (book2)
- "in" (book3)

To find the real differences, we're going to next look at the frequent 2-item sets. This should display some
differences between the three books and their writing styles.

The five most common two words for every book with a min-support of 0.02

In [230]:
book1_freq_items_1 = filter_freq_items(book1_freq_items, 2)
book1_freq_items_1.head(5)

,support,itemsets,length
177,0.138651,"(of, the)",2
105,0.098316,"(the, and)",2
190,0.095165,"(the, to)",2
106,0.092284,"(to, and)",2
101,0.089943,"(of, and)",2


In [231]:
book2_freq_items_1 = filter_freq_items(book2_freq_items, 2)
book2_freq_items_1.head(5)

,support,itemsets,length
151,0.171316,"(of, the)",2
91,0.163672,"(the, and)",2
160,0.109771,"(the, to)",2
138,0.105437,"(the, in)",2
88,0.092120,"(of, and)",2


In [232]:
book3_freq_items_1 = filter_freq_items(book3_freq_items, 2)
book3_freq_items_1.head(5)

,support,itemsets,length
162,0.311895,"(of, the)",2
135,0.164190,"(the, in)",2
88,0.163303,"(the, and)",2
181,0.126488,"(the, to)",2
84,0.126118,"(of, and)",2


### Frequent 2-itemsets

Right off the bat we can see some major differences by looking at the support values for the top 5 two itemset words.
 Book3, Darwin, is using a lot of two word itemsets much more. Book1 is showing a lot less support for using the same two
 word itemsets. Jane Austin's novel was probably a bit smoother of a read with more variety. To lay out the similarities
 in black and white below are the similarities and differences for the top 5.

 __Similarities__
 - "Of, The"
 - "The, And"
 - "The, To"

__Differences__
 - "To, And" (Book1, Book)

[Of, The] is naturally the fist for all three books. Looking at the 2 itemsets actually shows that all three authors
are using primarily the same sets of words. 